# Binomial Tree Model: Functions Using %'s for Up/Down Movement

In [2]:
import numpy as np
import math
import pandas as pd
import scipy.stats as st
from scipy.optimize import fsolve

def t_branch_binom_tree(t, p, r1, r2, S0):
    vals = []
    probabilities = []
    
    for i in range(0, t+1):
        vals.append((r1)**i * (r2)**(t-i)) # 'i' & 't-i' give the values required for multiplying the returns of each branch
        probabilities.append(st.binom.pmf(i, t, p)) # returns p_dist of each node using pmf
    
    vals = np.sort(np.array(vals)) # orders values in ascending order (decides the direction of the tree - if descending values then prob up = prob down)
        
    return S0*np.array(vals), np.array(probabilities)

def return_tree(t, p, r1, r2, S0):
    branches = []
    probabilities = []
    time = np.arange(t)
    
    for t in time:
        branches.append(t_branch_binom_tree(t, p, r1, r2, S0)[0])
        probabilities.append(t_branch_binom_tree(t, p, r1, r2, S0)[1])
    
    return pd.DataFrame(branches).fillna('-'), pd.DataFrame(probabilities).fillna('-')


# Generating Individual Branches of the Tree

In [77]:
S0 = 100 # initial value
t = 4 # set time steps
p = 0.5 # probabilities of up and down (up is p and down is q=1-p)
r1 = 1.05
r2 = 0.95

vals, probs = t_branch_binom_tree(t, p, r1, r2, S0)

branch_t = pd.DataFrame(np.array([vals, probs]).T, columns=['Value', 'Probability'])

print(branch_t)

        Value  Probability
0   81.450625       0.0625
1   90.024375       0.2500
2   99.500625       0.3750
3  109.974375       0.2500
4  121.550625       0.0625


# Generating Binomial Tree

In [78]:
S0 = 100 # initial value
t = 4 # set time steps
p = 0.5 # probabilities of up and down (up is p and down is q=1-p)
r1 = 1.05
r2 = 0.95

branches, probs = return_tree(t, p, r1, r2, S0)

print(branches, "\n", probs)

          0        1        2        3
0  100.0000        -        -        -
1   95.0000      105        -        -
2   90.2500    99.75   110.25        -
3   85.7375  94.7625  104.737  115.763 
        0      1      2      3
0  1.000      -      -      -
1  0.500    0.5      -      -
2  0.250    0.5   0.25      -
3  0.125  0.375  0.375  0.125


In [4]:
p = 0

In [6]:
import numpy as np
import math
import pandas as pd
import scipy.stats as st

S0 = 48 # initial value
K = 50
t = 2 # set time steps
#p = 0.5 # probabilities of up and down (up is p and down is q=1-p)
up = 3
down = 3
T = 0.5
r = 0.04
dt = T/t
O = 1

def f_delta(delta, O_up, S_up, O_down, S_down):
    return (O_up - S_up*delta) - (O_down - S_down*delta)

def f_option(O_curr, S_curr, delta, O_up, S_up, O_down, S_down, r, dt):
    return (O_curr - S_curr*delta) - np.exp(-r*(dt))*(O_down - S_down*delta)

def binomial_replication_pricing(S0, K, t, p, up, down, T, r, dt, O):
    # intialising numpy arrays for holding stock and option prices
    s_tree = np.array([[0] * (t+1) for i in range(t+1)]).astype(float)
    o_tree = np.array([[0] * (t+1) for i in range(t+1)]).astype(float)
    s_tree[0][0] = S0

    # obtaining stock prices for each branch
    for i in range(1, t+1): # each branch
        for j in range(i+1):
            if j != 0:
                s_tree[i][j] = s_tree[i-1][j-1] + up
            else:
                s_tree[i][j] = s_tree[i-1][j] - down

    # calculating the final row of option prices 
    o_tree[-1] = np.maximum(O*(s_tree[-1] - K), 0)

    # dynamically calculating options prices from T to current time
    for i in range(len(s_tree)-2, -1, -1):
        for j in range(0, i+1):
            delta_0 = 0.5 # guess
            delta = fsolve(f_delta, delta_0, (o_tree[i+1][j+1], s_tree[i+1][j+1], 
                                              o_tree[i+1][j], s_tree[i+1][j])) # solve
            O_0 = 1
            O_curr = fsolve(f_option, O_0, (s_tree[i][j], delta, o_tree[i+1][j+1], 
                                                  s_tree[i+1][j+1], o_tree[i+1][j], s_tree[i+1][j], 
                                                  r , dt)) # solve

            o_tree[i,j] = O_curr[0]


    return o_tree[0][0]

binomial_replication_pricing(S0, K, t, p, up, down, T, r, dt, O)

1.3322165477963919

In [162]:
o_tree[0][0] = float(O_curr[0])

print(o_tree)

[[1 0 0]
 [0 0 0]
 [4 0 0]]


# Trinomial Tree Model: Values & Probabilities of t'th branch

In [93]:
import numpy as np
import math
import pandas as pd
import scipy.stats as st

def ret_probabilities(t_branch, probability_set, tri_nom_vals):
    # storing all values of the t'th branch to 2dp
    all_vals = np.around(t_branch, decimals=2)

    # storing the unique values
    unique_vals = tri_nom_vals # unique values

    l = [] # np.array that will contain the index of the probabilities to be summed
    t_branch_probabilities = [] # list that will contain the final probabilities for the t'th branch

    # iterating through the unique branch values and saving their indices in the array
    for i in range(len(unique_vals)):
        l.append(np.where(all_vals == unique_vals[i]))

    # iterating through the array and taking the set of indices to sum and summing them in the probability set
    for j in range(len(l)):    
        idx = [l[j][0][i] for i in range(len(l[j][0]))]
        t_branch_probabilities.append(probability_set[idx].sum())
        
    return t_branch_probabilities


def trinom_tree(S0, t, sigma, p):
    r1 = math.exp(sigma * np.sqrt(1/t))
    r2 = 1/r1
    tree_branches = [np.array([S0])]
    probability_set = [np.array([(1)])]
    
    # iterates over t and multiplies out all probabilities and return values
    for i in range(t):
        tree_branches.append(np.concatenate((tree_branches[-1]*r1, tree_branches[-1], tree_branches[-1]*r2)))
        probability_set.append(np.concatenate((probability_set[-1]*p[0], probability_set[-1]*p[1], probability_set[-1]*p[2])))
        
    # identifies unique trinomial values and stores     
    tri_nom_vals = [np.unique(np.around(tree_branches[i], decimals=2)) for i in range(len(tree_branches))]
    
    # calls function ret_probabilities which identifies repeat values in t'th branch of tree_branch and then
    # finds their indices, and adds together the probabilities from probability set to calculate probabilities of each
    # output node
    tri_nom_probabilities = ret_probabilities(tree_branches[t], probability_set[t], tri_nom_vals[t])
    
    return np.array(tri_nom_vals[t]), np.array(tri_nom_probabilities)


In [110]:
# parameters to adjust
S0 = 100
t = 1
sigma = 0.11
p = [0.3, 0.4, 0.3]

# function call
v, p = trinom_tree(S0, t, sigma, p)

# formatting output
output = np.array([v, p]).T

# placing into DataFrame for clean output
print(pd.DataFrame(output, columns=['Value', 'Probability']))

# checking probabilities sum to 1
display(np.around(sum(p), decimals=2))

    Value  Probability
0   89.58          0.3
1  100.00          0.4
2  111.63          0.3


1.0